In [7]:
from pathlib import Path
import joblib

import pandas as pd

In [ ]:
def load_data(filename: str) -> pd.DataFrame:
    p = Path(f"./data/{filename}")
    assert p.exists()

    df = pd.read_csv(filepath_or_buffer=p)

    df["Genre"] = df.Genre.astype("category").cat.codes
    df["Episode_Sentiment"] = df.Episode_Sentiment.astype(
        "category"
    ).cat.codes
    df["Publication_Day"] = df.Publication_Day.astype("category").cat.codes
    df["Publication_Time"] = df.Publication_Time.astype(
        "category"
    ).cat.codes
    df["Episode_Title"] = df.Episode_Title.astype("category").cat.codes
    df["Podcast_Name"] = df.Podcast_Name.astype("category").cat.codes

    return df


test_df = load_data("test.csv")

In [2]:
modeldir = Path("./models")
model = modeldir / "rf_simple_model_bundle.pkl"

In [9]:
bundle = joblib.load(model)

In [12]:
model = bundle.get("model")

In [16]:
test_df["Listening_Time_minutes"] = model.predict(test_df)

In [19]:
submission_dir = Path("./submissions")
test_df.filter(items=["id", "Listening_Time_minutes"]).to_csv(
    submission_dir / "simple_rf.csv",
    index=False
)